# **Exercice 1 : Créez une chaîne LCEL qui génère une description simple d'un objet (par exemple, un fruit), puis reformule cette description en ajoutant une touche humoristique.**

**Énoncé :**
1. Définissez un prompt qui demande une description simple d'un objet, comme un fruit ou un légume, par exemple : "Décris-moi {objet}".
2. Chaînez ce prompt à un modèle de langage pour obtenir la description.
3. Ajoutez un second composant à la chaîne qui modifie cette description en ajoutant une phrase humoristique à la fin (par exemple, en utilisant une fonction personnalisée).
4. Invoquez la chaîne avec un exemple d’objet (ex. "pomme") et affichez la description humoristique finale.

**Étapes :**
- Créez un prompt LCEL qui accepte une variable {objet}.
Utilisez un modèle de langage (comme ChatOpenAI ou autre) pour générer la description.
- Ajoutez une fonction via RunnableLambda qui ajoute une phrase humoristique à la fin du texte généré.
- Exécutez la chaîne pour afficher la sortie humoristique.

In [13]:
!pip install -qU langchain_mistralai


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [14]:
import os
from dotenv import load_dotenv
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai import ChatMistralAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

In [15]:
load_dotenv()
mistral_key = os.getenv('MISTRAL_API_KEY')

In [16]:
model = ChatMistralAI(model="mistral-large-latest")
prompt = ChatPromptTemplate.from_template("Décris-moi simplement ce fruit ou légume : {objet}")
output_parser = StrOutputParser()

In [17]:
def ajouter_humour(texte: str) -> str:
    phrase_drole = "Et entre nous, il ferait un super super-héros dans un film Marvel…"
    return texte.strip() + " " + phrase_drole

humour_component = RunnableLambda(ajouter_humour)

In [18]:
chaine = prompt | model | output_parser | humour_component
resultat = chaine.invoke({"objet": "pomme"})
print(resultat)

La pomme est un fruit rond, généralement de couleur rouge, verte ou jaune, avec une peau lisse et brillante. Elle a une chair croquante et juteuse, et contient un noyau central avec des pépins. Les pommes peuvent être consommées crues, cuites ou utilisées dans diverses recettes comme les tartes, les compotes et les jus. Elles sont riches en vitamines et en fibres. Et entre nous, il ferait un super super-héros dans un film Marvel…


In [19]:
chaine2 = prompt | model | output_parser
resultat = chaine2.invoke({"objet": "pomme"})
print(resultat)

La pomme est un fruit rond, de couleur variable (rouge, verte, jaune ou une combinaison de ces couleurs), avec une peau lisse et brillante. Elle a une chair croquante et juteuse, souvent sucrée mais parfois acidulée selon la variété. Le centre de la pomme contient des pépins bruns. Elle est souvent consommée crue, mais peut aussi être cuisinée dans des tartes, compotes, ou jus.


# **Exercice 2 : Créez une chaîne LCEL qui exécute deux opérations en parallèle : une pour transformer une phrase en majuscules et une autre pour inverser l'ordre des mots de la même phrase.**

**Énoncé :**
1. Créez une fonction personnalisée qui met en majuscules une phrase donnée.
2. Créez une autre fonction qui inverse l'ordre des mots dans une phrase.
3. Utilisez RunnableParallel pour exécuter ces deux transformations simultanément.
4. Affichez les deux résultats, celui en majuscules et celui avec les mots inversés.

**Étapes :**
- Définissez une première fonction pour transformer une phrase en majuscules via RunnableLambda.
- Définissez une seconde fonction pour inverser les mots d’une phrase.
- Utilisez RunnableParallel pour exécuter ces deux fonctions simultanément.
- Invoquez la chaîne avec une phrase d’entrée et affichez les deux sorties.

In [20]:
import random
from langchain_core.runnables import RunnableLambda, RunnableParallel

In [21]:
def to_upper(text: str) -> str:
    """
    Transforme une chaîne de caractères en majuscules.
    Args:
        text (str): Le texte à transformer.
    Returns:
        str: Le texte transformé en majuscules.
    """
    return text.upper()

def reverse_words(text: str) -> str:
    """
    Mélange aléatoirement les mots d'une chaîne de caractères.
    Args:
        text (str): Le texte à traiter, composé de mots séparés par des espaces.
    Returns:
        str: Une nouvelle chaîne avec les mots dans un ordre aléatoire.
    """
    words = text.split()
    random.shuffle(words)  # Mélange la liste en place
    return " ".join(words)


# Création des Runnables
uppercase_chain = RunnableLambda(to_upper)
reverse_chain = RunnableLambda(reverse_words)

# Exécuter les deux opérations en parallèle
parallel_chain = RunnableParallel({
    "en_majuscules": uppercase_chain,
    "inversion_mots": reverse_chain
})

# Tester avec une phrase
input_text = "LangChain est vraiment puissant"
result = parallel_chain.invoke(input_text)

# Affichage des résultats
print("Résultat en majuscules :", result["en_majuscules"])
print("Résultat avec mots inversés :", result["inversion_mots"])


Résultat en majuscules : LANGCHAIN EST VRAIMENT PUISSANT
Résultat avec mots inversés : LangChain vraiment est puissant


# **Exercice 3 : Créez une chaîne LCEL qui pose une question à deux modèles de langage différents. Le premier modèle est configuré pour échouer, et en cas d’échec, la chaîne doit basculer vers un second modèle qui fonctionne.**

**Énoncé :**
1. Définissez un prompt qui pose une question générale, comme "Quelles sont les couleurs du drapeau italien ?".
2. Configurez deux modèles de langage :
- Le premier modèle doit être incorrect (simulez un échec en utilisant un modèle avec un nom incorrect).
- Le second modèle doit être configuré correctement pour fournir la réponse en cas d’échec du premier.
3. Utilisez with_fallbacks() pour définir le comportement en cas d’échec du premier modèle.
4. Testez la chaîne avec une question d’entrée et affichez la réponse fournie par le modèle fonctionnel.

**Étapes :**
- Créez un prompt simple qui accepte une question.
- Configurez un premier modèle incorrect pour provoquer une erreur.
- Configurez un second modèle qui fonctionne correctement.
- Créez une chaîne avec with_fallbacks() pour gérer l'échec du premier modèle et basculer sur le second.
- Exécutez la chaîne avec une question et vérifiez que la réponse correcte provient du second modèle en cas d’échec du premier.

In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_mistralai import ChatMistralAI

# Étape 1 : Créer un prompt simple
prompt = ChatPromptTemplate.from_messages([
    ("system", "Tu es un assistant utile."),
    ("human", "{question}")
])

# Étape 2 : Créer un modèle erroné (volontairement)
bad_model = ChatMistralAI(model="mistral-test-echec")  # faux nom

# Étape 3 : Créer un modèle fonctionnel
good_model = ChatMistralAI(model="mistral-large-latest")  # vrai modèle

# Étape 4 : Appliquer le fallback sur le modèle
fallback_model = bad_model.with_fallbacks([good_model])

# Étape 5 : Créer la chaîne complète
chain = prompt | fallback_model | StrOutputParser()

# Étape 6 : Tester
output = chain.invoke({"question": "Quelles sont les couleurs du drapeau italien ?"})
print("Réponse finale :", output)


Réponse finale : Le drapeau italien, également connu sous le nom de "Il Tricolore", est composé de trois bandes verticales de couleurs égales. De gauche à droite, les couleurs sont :

1. Vert
2. Blanc
3. Rouge

Ces couleurs symbolisent différentes valeurs et aspects de l'histoire italienne. Le vert représente les paysages et les plaines du pays, le blanc symbolise la neige des Alpes, et le rouge évoque le sang versé pendant les guerres d'indépendance et d'unification de l'Italie.
